# import_hook

所谓`import hook`就是指直接自定义finder和loader,并将finder放入`sys.meta_path`中的过程.

利用这个可以做到很多非常神奇的事情,比如

+ import某个特定模块时触发某个回调函数来通知我们
+ import一个远程服务器上的模块
+ 直接import其他语言(比如fortran)的模块来使用

本节需要的先验知识包括:

+ [模块的导入方式]()
+ [使用f2py为python嵌入fortran代码]()


## import hook的基本形式

import hook通常是以一个单文件模块的形式出现的,其中的过程说白了就是自定义finder和loader,因此自定义这两个类都是必须的,然后就是将定义的finder实例化,并将这个实例加入`sys.meta_path`.下面是模板代码.

```python
import importlib
from importlib.abc import (
    MetaPathFinder, 
    PathEntryFinder,
    Loader
)
from importlib.machinery import ModuleSpec
import sys
from collections import defaultdict


class ClientImportLoader(Loader):
    @classmethod
    def create_module(clz,spec):
        """用于创建模块的."""
        module = __create_module_from_spec(spec)
        return module or None

    @classmethod
    def exec_module (clz, module):
        """每次执行引入模块或者重载模块时会执行的操作"""
        pass

loader= ClientImportLoader()
    

class ClientImportFinder(MetaPathFinder):

    @classmethod
    def find_spec (klass, full_name, paths=None, target=None):
        """查找模块的逻辑"""
        pass
        return ModuleSpec(full_name, loader, origin=module_full_path)
    

sys.meta_path.insert(0, ClientImportFinder())

```


当这个定义import hook的模块被加载后,他就可以正常的执行自己的功能了,因此通常这个import hook的模块需要优先加载.


## import某个特定模块时触发某个回调函数来通知我们

这个例子来自python cookbook,不过上面的代码已经比较过时了,这边给出python3.5+推荐的写法

In [1]:
import importlib
from importlib.abc import (
    MetaPathFinder, 
    PathEntryFinder,
    Loader
)
from importlib.machinery import ModuleSpec
import sys
from collections import defaultdict

_post_import_hooks = defaultdict(list)

class ClientImportLoader(Loader):
    def __init__(self, finder):
        self._finder = finder
        

    def create_module(self,spec):
        """这边只要调用父类的实现即可."""
        return super().create_module(spec)

    def exec_module (self, module):
        """在_post_import_hooks中查找对应模块中的回调函数并执行."""
        for func in _post_import_hooks[module.__name__]:
            func(module)
        self._finder._skip.remove(module.__name__)
        
class ClientImportFinder(MetaPathFinder):
    
    def __init__(self):
        self._skip = set()

    def find_spec (self, full_name, paths=None, target=None):
        """."""
        if full_name in self._skip:
            return None
        self._skip.add(full_name)
        loader = ClientImportLoader(self)
        return ModuleSpec(full_name, loader, origin=paths)
        
        
def when_imported(fullname):
    def decorate(func):
        if fullname in sys.modules:
            func(sys.modules[fullname])
        else:
            _post_import_hooks[fullname].append(func)
        return func
    return decorate

finder = ClientImportFinder()
sys.meta_path.insert(0, finder)

In [2]:
@when_imported('numpy')
def warn_numpy(mod):
    print('numpy? Are you crazy?')


In [3]:
import numpy

numpy? Are you crazy?


In [4]:
finder._skip

set()

为了避免陷入无线循环,ClientImportFinder维护了一个所有被加载过的模块集合`_skip`,如果一个模块在加载过程中又有另一个地方来加载,那么就会跳过这个加载器

## import一个远程服务器上的模块


这个例子主要是复写finder以可以查找到目标服务器上的模块文件.同时复写loader的create_module方法用远端的代码生成服务.

我们的远程代码以http服务的形式放在静态服务器上
```shell
testcode-|
         |-spam.py
         |-fib.py
         |-grok-|
                |-__init__.py
                |-blah.py
```

`spam.py`
```python
print("I'm spam")

def hello(name):
    print('Hello %s' % name)
```

`fib.py`
```python
print("I'm fib")

def fib(n):
    if n < 2:
        return 1
    else:
        return fib(n-1) + fib(n-2)
```

`grok/__init__.py`

```python
print("I'm grok.__init__")
```
`grok/blah.py`

```python
print("I'm grok.blah")
```
使用python自带的http服务启动:

```shell
cd source/testcode
python3 -m http.server 15000
```

In [8]:
import requests
n = requests.get("http://localhost:15000/fib.py")
print(n.content.decode("utf-8"))

print("I'm fib")

def fib(n):
    if n < 2:
        return 1
    else:
        return fib(n-1) + fib(n-2)


### 最简单的方法

我们可以使用`imp.new_module`新建一个空的模块对象,再使用内置方法`compile()`将源码编译到一个代码对象中,然后在模块对象的字典中来执行它.
这种方式没有嵌入到通常的import语句中,如果要支持更高级的结构比如包就需要更多的工作了.

下面是使用这个函数的方式:

In [10]:
import imp
import urllib.request
import sys

def load_module(url):
    u = urllib.request.urlopen(url)
    source = u.read().decode('utf-8')
    mod = sys.modules.setdefault(url, imp.new_module(url))
    code = compile(source, url, 'exec')
    mod.__file__ = url
    mod.__package__ = ''
    exec(code, mod.__dict__)
    return mod

In [11]:
fib = load_module('http://localhost:15000/fib.py')

I'm fib


In [12]:
fib.fib(10)

89

In [13]:
fib

<module 'http://localhost:15000/fib.py' from 'http://localhost:15000/fib.py'>